In [15]:
import pandas as pd
import numpy as np
import datetime as dt
import time
import random
from tqdm import tqdm
import urllib2
import requests
from geopy.geocoders import Nominatim
import json 
import os

# Moving historic data

In [39]:
directory = 'blob/full_data/'
destination = 'blob/historic/'
dirs = os.listdir(directory)

for fl in tqdm(dirs):
    try:
        dest = open(destination + 'darksky_historic_' + fl.split('_')[2], 'w+')
        src = open(directory + fl, 'r')
        json_src = json.load(src)
        #dest.write(json.dumps(json_src["daily"], indent=4))
        dest.write(json.dumps(json_src, indent=4))

        src.close()
        dest.close()
    except:
        print fl

 13%|█▎        | 99/752 [00:00<00:02, 323.58it/s]

darksky_full_1430172000.json


100%|██████████| 752/752 [00:02<00:00, 329.69it/s]


In [67]:
directory = 'blob/parsed_data/'
destination = 'blob/'
dirs = os.listdir(directory)
dest = open(destination + 'darksky_combined_1.json', 'w+')
counter = 0

for fl in tqdm(dirs):
    src = open(directory + fl, 'r')
    json_src = json.load(src)
    #dest.write(json.dumps(json_src["daily"], indent=4))
    dest.write(json.dumps(json_src, indent=4))

    src.close()
    counter += 1

    if(counter % 500 == 0):
        dest.close()
        dest = open(destination + 'darksky_combined_' + str(counter/500 + 1) + '.json' , 'w+')
        
   
dest.close()

  0%|          | 0/746 [00:00<?, ?it/s]


ValueError: Extra data: line 31 column 2 - line 721 column 2 (char 1148 - 27697)

# Parsing the data

In [44]:
def f_t_c(fahrenheit):
    return (((fahrenheit - 32) * 5.0) / 9.0)

In [90]:
directory = 'blob/historic/'
destination = 'blob/parsed_data/'
dirs = os.listdir(directory)
alldata = []

for fl in tqdm(dirs):
    try:
        src = open(directory + fl, 'r')
        dest = open(destination + 'darksky_parsed_' + fl.split("_")[2], 'w+')
        json_src = json.load(src)  
        json_temp = json_src.get('hourly').get('data')


        for temp in json_temp:
            data = {}
            #changing hourly keys
            data["HourlyTimestamp"] = dt.datetime.fromtimestamp(temp["time"]).strftime('%Y-%m-%d %H:%M:%S')
            data["HourlyTemperature"] = f_t_c(temp["temperature"])
            data["HourlyDewPoint"] = temp["dewPoint"]
            data["HourlySummary"] = temp["summary"]
            data["HourlyApparentTemperature"] = f_t_c(temp["apparentTemperature"])
            try:
                data["HourlyPressure"] = temp["pressure"]
            except KeyError as e:
                data["HourlyPressure"] = -1.0
            try:
                data["HourlyWindSpeed"] = temp["windSpeed"] 
            except KeyError as e:
                data["HourlyWindSpeed"] = -1.0
            try:
                data["HourlyWindBearing"] = temp["windBearing"]
            except KeyError as e:
                data["HourlyWindBearing"] = -1
            
            data["HourlyHumidity"] = temp["humidity"]
            data["HourlyIcon"] = temp["icon"]

            #appending daily data
            data["DailyTemperatureMax"] = f_t_c(json_src.get("daily").get('data')[0]["temperatureMax"])
            data["DailyApparentTemperatureMinTime"] = json_src.get("daily").get('data')[0]["apparentTemperatureMinTime"]
            data["DailyTemperatureMin"] = f_t_c(json_src.get("daily").get('data')[0]["temperatureMin"])
            data["DailySummary"] = json_src.get("daily").get('data')[0]["summary"]
            data["DailyDewPoint"] = json_src.get("daily").get('data')[0]["dewPoint"]
            data["DailyApparentTemperatureMax"] = f_t_c(json_src.get("daily").get('data')[0]["apparentTemperatureMax"])
            data["DailyTemperatureMaxTime"] = json_src.get("daily").get('data')[0]["temperatureMaxTime"]
            data["DailyWindBearing"] = json_src.get("daily").get('data')[0]["windBearing"]
            data["DailyMoonPhase"] = json_src.get("daily").get('data')[0]["moonPhase"]
            data["DailySunsetTime"] = json_src.get("daily").get('data')[0]["sunsetTime"]
            try:
                data["DailyPressure"] = json_src.get("daily").get('data')[0]["pressure"]
            except KeyError as e:
                data["DailyPressure"] = -1.0
            data["DailyApparentTemperatureMin"] = f_t_c(json_src.get("daily").get('data')[0]["apparentTemperatureMin"])
            data["DailyIcon"] = json_src.get("daily").get('data')[0]["icon"]
            data["DailyapparentTemperatureMaxTime"] = json_src.get("daily").get('data')[0]["apparentTemperatureMaxTime"]
            data["DailyHumidity"] = json_src.get("daily").get('data')[0]["humidity"]
            try:
                data["DailyWindSpeed"] = json_src.get("daily").get('data')[0]["windSpeed"]
            except KeyError as e:
                data["DailyWindSpeed"] = -1.0
            data["DailyDate"] = dt.datetime.fromtimestamp(json_src.get("daily").get('data')[0]["time"]).strftime('%Y-%m-%d %H:%M:%S')
            data["DailySunriseTime"] = json_src.get("daily").get('data')[0]["sunriseTime"]
            data["DailyTemperatureMinTime"] = json_src.get("daily").get('data')[0]["temperatureMinTime"]

            dest.write(json.dumps(data, indent=4) + "\n")
            alldata.append(data)
    except IOError as ioe:
        print "Combined folder found :p"




    #dest.write(json.dumps(json_src["daily"], indent=4))


    src.close()
    dest.close()


 67%|██████▋   | 503/747 [00:03<00:01, 167.10it/s]

Combined folder found :p


100%|██████████| 747/747 [00:04<00:00, 161.28it/s]


In [80]:
print alldata[0]

{'DailyapparentTemperatureMaxTime': 1480510800, 'DailyApparentTemperatureMinTime': 1480467600, 'HourlyTemperature': -3.483333333333333, 'DailyHumidity': 0.76, 'DailyWindBearing': 222, 'DailyApparentTemperatureMax': 1.772222222222221, 'HourlyDewPoint': 23, 'HourlyApparentTemperature': -3.483333333333333, 'DailySunsetTime': 1480520442, 'DailyTemperatureMinTime': 1480474800, 'DailyDewPoint': 24.37, 'HourlyTimestamp': '2016-11-30 00:00:00', 'HourlyPressure': 1035.24, 'HourlyWindSpeed': 1.1, 'DailyTemperatureMin': -3.922222222222222, 'HourlySummary': u'Clear', 'DailyTemperatureMax': 4.449999999999999, 'HourlyWindBearing': 308, 'DailyTemperatureMaxTime': 1480510800, 'DailyIcon': u'partly-cloudy-night', 'HourlyHumidity': 0.89, 'DailyPressure': 1034.21, 'DailySummary': u'Partly cloudy throughout the day.', 'DailyMoonPhase': 0.03, 'DailyApparentTemperatureMin': -7.322222222222223, 'DailySunriseTime': 1480490535, 'DailyDate': '2016-11-30 00:00:00', 'HourlyIcon': u'clear-night', 'DailyWindSpeed':

In [92]:
dest = open('darksky_combined_1.json', 'w+')
counter = 0

for datapoint in tqdm(alldata):
    
    dest.write(json.dumps(datapoint) + "\n")

    counter += 1

    if(counter % 9000 == 0):
        dest.close()
        dest = open('darksky_combined_' + str(counter/9000 + 1) + '.json' , 'w+')
        
   
dest.close()

100%|██████████| 17904/17904 [00:00<00:00, 20482.15it/s]


In [87]:
len(alldata)/2

8952